In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt 
import statistics as st
from datetime import datetime
from math import ceil, pi

In [ ]:
def LowPassFilter(array, sample_rate, frequency):
 dt = 1 / sample_rate
 RC = 1 / (2 * 3.14 * frequency)
 alpha = dt / (RC + dt)
 size = len(array)
 new_array = np.empty(size)
 for i in range(size):
  new_array[i] = alpha * array[i] + (1 - alpha) * new_array[i - 1]
 return new_array

def NormalizeArray(array):
 norm = np.linalg.norm(array)
 normal_array = array/norm
 return normal_array

def ValuesStd(error_max, array_x, array_y, approach):
 error = ceil(error_max)
 x = array_x
 y = array_y
 result = np.empty(error)
 for i in range(error):
  if len(x) == len(y):
   size = len(x)
  else:
   return np.empty(0)
  abs_values = np.empty(size)
  for j in range(size):
   abs_values[j] = abs(x[j] - y[j])
  result[i] = st.stdev(abs_values)
  if approach == False:
   x = x[0:(size - 1)]
   y = y[1:size]
  else:
   y = y[0:(size - 1)]
   x = x[1:size]
 return result

def FirstLocalMinimum(array, error_max):
 for i in range(ceil(error_max) - 1):
  if i != 0 and array[i] < array[i - 1] and array[i] < array[i + 1]:
   return i
 return error_max

def Sincro(x, y, error_max): 
 x = LowPassFilter(x, 1, pi)
 y = LowPassFilter(y, 1, pi)
 x = NormalizeArray(x)
 y = NormalizeArray(y)
 approach_neg = ValuesStd(error_max, x, y, False)
 approach_pos = ValuesStd(error_max, x, y, True)
 approach_neg_minimum = FirstLocalMinimum(approach_neg, error_max)
 approach_pos_minimum = FirstLocalMinimum(approach_pos, error_max)
 return -approach_neg_minimum if approach_neg_minimum <= approach_pos_minimum else approach_pos_minimum

In [ ]:
n = range(10)
theta = (np.pi / 2) * np.array(n)
npx = np.cos(theta)
npy = np.sin(theta)
plt.plot(range(len(n)), npx, npy)
plt.show()
sincro_value = Sincro(npx,npy,9)
print("Sicro: ", sincro_value)

In [ ]:
plt.figure()
plt.figure(figsize=(10, 5), dpi=80)
plt.subplot(121)
plt.plot(range(len(n)-1), npx[0:len(n)-1], npy[1:len(n)])
plt.subplot(122)
plt.plot(range(len(n)-1), npx[1:len(n)], npy[0:len(n)-1])
plt.show()

In [ ]:
data = pd.read_csv("/content/drive/MyDrive/Lynx/dataset_plc.csv")
str_speed_ideal = data.loc[data['Variavel'] == 'PLC_WRITE_RP313K06_INTERS_3D_MAX_SPEED']
str_speed_feedback = data.loc[data['Variavel'] == 'PLC_READ_RP313K06_FEEDBACK_SPEED_SLEWING']
values_speed_ideal = str_speed_ideal['Valor'].values.astype(np.float)
values_speed_feedback = str_speed_feedback['Valor'].values.astype(np.float)

In [ ]:
size_entries = len(values_speed_ideal) if len(values_speed_ideal) < len(values_speed_feedback) else len(values_speed_feedback)
print("Size of the smallest vector: ", size_entries)

#plt.figure()
#plt.figure(figsize=(15, 10), dpi=80)
#plt.subplot(231)
#plt.plot(values_speed_ideal[0:size_entries])
#plt.subplot(232)
#plt.plot(values_speed_feedback[0:size_entries])
#plt.subplot(233)
#plt.plot(abs(values_speed_feedback)[0:size_entries])
#plt.subplot(234)
#plt.plot(range(size_entries), values_speed_ideal[0:size_entries], abs(values_speed_feedback)[0:size_entries])
#plt.subplot(235)
#plt.plot(values_speed_ideal[0:size_entries], abs(values_speed_feedback)[0:size_entries], 'o')
#plt.show()

values_speed_ideal = values_speed_ideal[0:size_entries]
values_speed_feedback = values_speed_feedback[0:size_entries]

In [ ]:
x = values_speed_ideal
y = values_speed_feedback

i = 0
size_plot = 500

plt.figure()
plt.figure(figsize=(30, 5))
plt.plot(range(size_plot-i), x[i:size_plot], y[i:size_plot])
plt.show()

In [ ]:
data = pd.read_csv("/content/drive/MyDrive/Lynx/dataset_plc.csv")

str_speed_max = data.loc[data['Variavel'] == 'PLC_WRITE_RP313K06_INTERS_3D_MAX_SPEED']
str_speed_feedback = data.loc[data['Variavel'] == 'PLC_READ_RP313K06_FEEDBACK_SPEED_SLEWING']

d_speed_max = {'DateTime': str_speed_max['DateTime'].values.astype(datetime), 'Valor': str_speed_max['Valor'].values.astype(np.float)}
d_speed_feedback = {'DateTime': str_speed_feedback['DateTime'].values.astype(datetime), 'Valor': str_speed_feedback['Valor'].values.astype(np.float)}

speed_max = pd.DataFrame(data=d_speed_max)
speed_feedback = pd.DataFrame(data=d_speed_feedback)

for i in range(len(speed_max['DateTime'])):
 speed_max.loc[i, 'DateTime'] = datetime.strptime(speed_max['DateTime'][i], '%d/%m/%Y %H:%M:%S')

for i in range(len(speed_feedback['DateTime'])):
 speed_feedback.loc[i, 'DateTime'] = datetime.strptime(speed_feedback['DateTime'][i], '%d/%m/%Y %H:%M:%S')

In [ ]:
plt.figure()
plt.figure(figsize=(30, 10))
plt.subplot(211)
plt.plot(speed_max['DateTime'], speed_max['Valor'], speed_feedback['DateTime'], abs(speed_feedback['Valor']))
plt.subplot(212)
plt.plot(speed_max['DateTime'], speed_max['Valor'], 'o', speed_feedback['DateTime'], abs(speed_feedback['Valor']), '+')
plt.show()